In [1]:
import pandas as pd
import requests
from pandas_profiling import ProfileReport

In [3]:
response = requests.get("https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv")
response

<Response [200]>

In [4]:
response = response.content.decode("utf-8")
result = [x.split(',') for x in response.split('\n')]
df = pd.DataFrame(result[1:], columns= result[0])

In [5]:
df

,epi_week,date,country,state,city,newDeaths,deaths,newCases,totalCases,deathsMS,...,tests,tests_per_100k_inhabitants,vaccinated,vaccinated_per_100_inhabitants,vaccinated_second,vaccinated_second_per_100_inhabitants,vaccinated_single,vaccinated_single_per_100_inhabitants,vaccinated_third,vaccinated_third_per_100_inhabitants
0,9,2020-02-25,Brazil,SP,TOTAL,0,0,1,1,0,...,,,,,,,,,,
1,9,2020-02-25,Brazil,TOTAL,TOTAL,0,0,1,1,0,...,,,,,,,,,,
2,9,2020-02-26,Brazil,SP,TOTAL,0,0,0,1,0,...,,,,,,,,,,
3,9,2020-02-26,Brazil,TOTAL,TOTAL,0,0,0,1,0,...,,,,,,,,,,
4,9,2020-02-27,Brazil,SP,TOTAL,0,0,0,1,0,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27226,245,2022-11-09,Brazil,SE,TOTAL,0,6443,0,342918,6443,...,604183,26283.72782,1992913,86.69755,1834937,79.82513,42285,1.83952,1376006,59.86029
27227,245,2022-11-09,Brazil,SP,TOTAL,0,175673,0,6149751,175673,...,19501783,42469.91918,42751507,93.10190,40221220,87.59158,1280044,2.78761,27899204,60.75736
27228,245,2022-11-09,Brazil,TO,TOTAL,0,4206,0,344864,4206,...,903975,57473.10960,1167326,74.21649,1007278,64.04093,56219,3.57430,526358,33.46490
27229,245,2022-11-09,Brazil,TOTAL,TOTAL,0,688395,0,34855492,688395,...,70923215,33247.70297,181964720,85.30224,166908532,78.24413,5434803,2.54775,118811893,55.69717


In [ ]:
df.info()

In [ ]:
df.date.unique()

In [6]:
profile = ProfileReport(df, title="Relatório de Empréstimo Pessoal", explorative = True)

In [ ]:
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]